In [7]:
import os
import numpy
import statsmodels
import statsmodels.api
import pandas as pd
from google.cloud import bigquery as bq
from google.oauth2 import service_account

credentials = service_account.Credentials.from_service_account_file("/Users/jacktoke/.gcp_keys/qut-data-analytics-capstone-e8b361c29204.json")
project_id = os.environ["GOOGLE_CLOUD_PROJECT"]

client = bq.Client(credentials=credentials, project=project_id)



In [16]:
query = """
  WITH monthly_traps_data AS (
    SELECT
      STRIKED_YEAR_MONTH,
      STRIKED_MONTH,
      STRIKED_YEAR,
      NUM_KILLS,
      NUM_TRAPS
    FROM
      `goodnature.fct_nz_monthly_kills`
  )
    SELECT
      STRIKED_YEAR_MONTH,
      STRIKED_MONTH,
      STRIKED_YEAR,
      SUM(NUM_KILLS) TOTAL_KILLS,
      SUM(NUM_TRAPS) NUM_TRAPS
    FROM monthly_traps_data
    GROUP BY STRIKED_YEAR_MONTH, STRIKED_YEAR, STRIKED_MONTH
    ORDER BY STRIKED_YEAR, STRIKED_MONTH;
"""
query_job = client.query(query=query)
results_df = query_job.to_dataframe()

results_df.to_csv("data/monthly_stats.csv")

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning: BigQuery Storage module not found, fetch data with the REST endpoint instead.
  warnings.warn(


In [10]:
from scipy.stats import pearsonr

correlation, p_value = pearsonr(results_df["NUM_TRAPS"], results_df["TOTAL_KILLS"])
print(f"Pearson Correlation Coefficient: {correlation}")
print(f"P-value: {p_value}")

Pearson Correlation Coefficient: 0.9821733832523392
P-value: 1.673422158466051e-36


In [11]:
x_bar = results_df["NUM_TRAPS"].mean()
print(f"x_bar_num_traps: {x_bar}")

x_bar_num_traps: 910.3


In [12]:
y_bar = results_df["TOTAL_KILLS"].mean()
print(f"y_bar_num_kills: {y_bar}")

y_bar_num_kills: 2099.02


In [ ]:
results_df["x_diff"] = results_df["NUM_TRAPS"]-x_bar
results_df["y_diff"] = results_df["TOTAL_KILLS"]-y_bar

results_df["x_e_time_y_e"] = results_df["x_diff"]*results_df["y_diff"]
results_df["x_diff_square"] = results_df["x_diff"]**2
results_df["y_diff_square"] = results_df["y_diff"]**2
results_df

,STRIKED_YEAR_MONTH,STRIKED_MONTH,STRIKED_YEAR,TOTAL_KILLS,NUM_TRAPS,x_diff,y_diff,x_e_time_y_e,x_diff_square,y_diff_square
0,2017-04,04,2017,200,29,-881.3,-1899.02,1673606.326,776689.69,3606276.9604
1,2017-05,05,2017,224,34,-876.3,-1875.02,1643080.026,767901.69,3515700.0004
2,2017-06,06,2017,753,133,-777.3,-1346.02,1046261.346,604195.29,1811769.8404
3,2017-07,07,2017,307,68,-842.3,-1792.02,1509418.446,709469.29,3211335.6804
4,2017-08,08,2017,13,1,-909.3,-2086.02,1896817.986,826826.49,4351479.4404
5,2017-09,09,2017,767,27,-883.3,-1332.02,1176573.266,780218.89,1774277.2804
6,2017-10,10,2017,903,160,-750.3,-1196.02,897373.806,562950.09,1430463.8404
7,2017-11,11,2017,318,54,-856.3,-1781.02,1525087.426,733249.69,3172032.2404
8,2017-12,12,2017,167,39,-871.3,-1932.02,1683369.026,759163.69,3732701.2804
9,2018-01,01,2018,205,81,-829.3,-1894.02,1570710.786,687738.49,3587311.7604


In [15]:
import math
sum_x_diff_time_y_diff = results_df["x_e_time_y_e"].sum()
sqrt_sum_x_diff_square_time_y_diff_square = math.sqrt(results_df["x_diff_square"].sum()*results_df["y_diff_square"].sum())
sqrt_sum_x_diff_square_time_y_diff_square

89018939.21262683

In [17]:
r = sum_x_diff_time_y_diff/sqrt_sum_x_diff_square_time_y_diff_square
r

np.float64(0.982173383252339)

In [18]:
r**2

np.float64(0.9646645547693461)

In [ ]:
query_2 = """
WITH ranked_stats AS (SELECT
DENSE_RANK() OVER(PARTITION BY YEAR ORDER BY NUM_KILLS DESC, NUM_TRAPS DESC, TERRITORIAL_AUTHORITY) AS RANK_ORDER,
*
FROM `goodnature.fct_nz_annual_stats`
)
SELECT
RANK_ORDER,
YEAR,
TERRITORY_AUTHORITY_ID,
TERRITORIAL_AUTHORITY,
NUM_SUBURBS,
TOTAL_NUM_SUBURBS,
SUBURB_COVERAGE_PERCENTAGE,
NUM_TOWNS,
TOTAL_NUM_TOWNS,
TOWN_COVERAGE_PERCENTAGE,
NUM_KILLS,
NUM_TRAPS
FROM ranked_stats
WHERE RANK_ORDER <= 10
ORDER BY YEAR, RANK_ORDER ASC
"""

query_job = client.query(query=query_2)
yearly_stats = query_job.to_dataframe()
yearly_stats

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,RANK_ORDER,YEAR,TERRITORY_AUTHORITY_ID,TERRITORIAL_AUTHORITY,NUM_SUBURBS,TOTAL_NUM_SUBURBS,SUBURB_COVERAGE_PERCENTAGE,NUM_TOWNS,TOTAL_NUM_TOWNS,TOWN_COVERAGE_PERCENTAGE,NUM_KILLS,NUM_TRAPS
0,1,2017,47,Wellington City,27,58,46.551724,1,2,50.000000,1239,117
1,2,2017,76,Auckland,37,304,12.171053,14,52,26.923077,550,88
2,3,2017,11,Thames-Coromandel District,6,60,10.000000,3,9,33.333333,515,142
3,4,2017,57,Westland District,1,32,3.125000,1,14,7.142857,322,2
4,5,2017,21,Taupo District,3,55,5.454545,3,10,30.000000,136,9
5,6,2017,63,Ashburton District,1,50,2.000000,1,6,16.666667,118,16
6,7,2017,70,Queenstown-Lakes District,3,51,5.882353,1,9,11.111111,67,7
7,8,2017,22,Western Bay of Plenty District,4,33,12.121212,3,24,12.500000,65,13
8,9,2017,53,Marlborough District,5,99,5.050505,2,19,10.526316,58,9
9,10,2017,51,Tasman District,8,91,8.791209,6,16,37.500000,53,14


In [50]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=5, cols=1, row_heights=[300, 300, 300, 300, 300], shared_yaxes=True,
                    subplot_titles=["2017 Top Ten Kills Territories",
                                    "2018 Top Ten Kills Territories",
                                    "2019 Top Ten Kills Territories",
                                    "2020 Top Ten Kills Territories",
                                    "2021 Top Ten Kills Territories", ],
                    y_title="Number of Kills/Traps")
# 2017
year_2017_stats = yearly_stats[yearly_stats["YEAR"]=="2017"]
fig.add_trace(
    go.Bar(x=year_2017_stats.TERRITORIAL_AUTHORITY, y=year_2017_stats.NUM_KILLS, text=year_2017_stats.NUM_KILLS,
           name="2017 #Kills"), 1, 1)
fig.add_trace(
    go.Bar(x=year_2017_stats.TERRITORIAL_AUTHORITY, y=year_2017_stats.NUM_TRAPS, text=year_2017_stats.NUM_TRAPS,
           name="2017 #Traps"), 1, 1)

year_2018_stats = yearly_stats[yearly_stats["YEAR"] == "2018"]
fig.add_trace(
    go.Bar(x=year_2018_stats.TERRITORIAL_AUTHORITY, y=year_2018_stats.NUM_KILLS, text=year_2018_stats.NUM_KILLS,
           name="2018 #Kills", ), 2, 1
)
fig.add_trace(
    go.Bar(x=year_2018_stats.TERRITORIAL_AUTHORITY, y=year_2018_stats.NUM_TRAPS, text=year_2018_stats.NUM_TRAPS,
           name="2018 #Traps", ), 2, 1
)

year_2019_stats = yearly_stats[yearly_stats["YEAR"]=="2019"]
fig.add_trace(
    go.Bar(x=year_2019_stats.TERRITORIAL_AUTHORITY, y=year_2019_stats.NUM_KILLS, text=year_2019_stats.NUM_KILLS,
           name="2019 #Kills"), 3, 1
)
fig.add_trace(
    go.Bar(x=year_2019_stats.TERRITORIAL_AUTHORITY, y=year_2019_stats.NUM_TRAPS, text=year_2019_stats.NUM_TRAPS,
           name="2019 #Traps"), 3, 1
)

year_2020_stats = yearly_stats[yearly_stats["YEAR"]=="2020"]
fig.add_trace(
    go.Bar(x=year_2020_stats.TERRITORIAL_AUTHORITY, y=year_2020_stats.NUM_KILLS, text=year_2020_stats.NUM_KILLS,
           name="2020 #Kills"), 4, 1
)
fig.add_trace(
    go.Bar(x=year_2020_stats.TERRITORIAL_AUTHORITY, y=year_2020_stats.NUM_TRAPS, text=year_2020_stats.NUM_TRAPS,
           name="2020 #Traps"), 4, 1
)

year_2021_stats = yearly_stats[yearly_stats["YEAR"]=="2021"]
fig.add_trace(
    go.Bar(x=year_2021_stats.TERRITORIAL_AUTHORITY, y=year_2021_stats.NUM_KILLS, text=year_2021_stats.NUM_KILLS,
           name="2021 #Kills"), 5, 1
)
fig.add_trace(
    go.Bar(x=year_2021_stats.TERRITORIAL_AUTHORITY, y=year_2021_stats.NUM_TRAPS, text=year_2021_stats.NUM_TRAPS,
           name="2021 #Traps"), 5, 1
)

fig.update_layout(
    height=2000,
    barmode="group",
    title_text="Top Ten Territories/District by #Kills vs #Traps",
)

fig.show()

In [ ]:
yearly_kills = yearly_stats.pivot_table(
    values=["NUM_KILLS", "NUM_TRAPS", "NUM_TOWNS", "TOTAL_NUM_TOWNS", "NUM_SUBURBS", "TOTAL_NUM_SUBURBS"],
    index="TERRITORIAL_AUTHORITY",
    columns="YEAR",
    )

Pivot Table (Sum of Sales):
Product    A    B
Region           
East     220  130
West     180  310


In [29]:
year_2017_stats = yearly_stats[yearly_stats["YEAR"]=="2017"]
year_2017_stats

,YEAR,TERRITORY_AUTHORITY_ID,TERRITORIAL_AUTHORITY,NUM_SUBURBS,TOTAL_NUM_SUBURBS,SUBURB_COVERAGE_PERCENTAGE,NUM_TOWNS,TOTAL_NUM_TOWNS,TOWN_COVERAGE_PERCENTAGE,NUM_KILLS,NUM_TRAPS
0,2017,47,Wellington City,27,58,46.551724,1,2,50.000000,1239,117
1,2017,76,Auckland,37,304,12.171053,14,52,26.923077,550,88
2,2017,11,Thames-Coromandel District,6,60,10.000000,3,9,33.333333,515,142
3,2017,57,Westland District,1,32,3.125000,1,14,7.142857,322,2
4,2017,21,Taupo District,3,55,5.454545,3,10,30.000000,136,9
...,...,...,...,...,...,...,...,...,...,...,...
61,2017,29,Wairoa District,0,16,0.000000,0,9,0.000000,0,0
62,2017,68,Waitaki District,0,95,0.000000,0,15,0.000000,0,0
63,2017,20,Waitomo District,0,24,0.000000,0,9,0.000000,0,0
64,2017,25,Whakatane District,0,38,0.000000,0,14,0.000000,0,0


In [37]:
year_2017_stats.TERRITORIAL_AUTHORITY

0                Wellington City
1                       Auckland
2     Thames-Coromandel District
3              Westland District
4                 Taupo District
                 ...            
61               Wairoa District
62              Waitaki District
63              Waitomo District
64            Whakatane District
65              Ōpōtiki District
Name: TERRITORIAL_AUTHORITY, Length: 66, dtype: object

West Land

In [ ]:
query_3 = """
WITH ranked_stats AS (SELECT
*
FROM `goodnature.fct_nz_annual_stats`
WHERE TERRITORIAL_AUTHORITY = 'Westland District'
)
SELECT
YEAR,
TERRITORY_AUTHORITY_ID,
TERRITORIAL_AUTHORITY,
NUM_SUBURBS,
TOTAL_NUM_SUBURBS,
SUBURB_COVERAGE_PERCENTAGE,
NUM_TOWNS,
TOTAL_NUM_TOWNS,
TOWN_COVERAGE_PERCENTAGE,
NUM_KILLS,
NUM_TRAPS
FROM ranked_stats
ORDER BY YEAR
"""

query_job3 = client.query(query=query_3)
westland_stats = query_job3.to_dataframe()
westland_stats

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,YEAR,TERRITORY_AUTHORITY_ID,TERRITORIAL_AUTHORITY,NUM_SUBURBS,TOTAL_NUM_SUBURBS,SUBURB_COVERAGE_PERCENTAGE,NUM_TOWNS,TOTAL_NUM_TOWNS,TOWN_COVERAGE_PERCENTAGE,NUM_KILLS,NUM_TRAPS
0,2017,57,Westland District,1,32,3.125,1,14,7.142857,322,2
1,2018,57,Westland District,1,32,3.125,1,14,7.142857,3,2
2,2019,57,Westland District,6,32,18.750,4,14,28.571429,32,28
3,2020,57,Westland District,12,32,37.500,5,14,35.714286,174,159
4,2021,57,Westland District,11,32,34.375,7,14,50.000000,163,154


In [54]:
westland_kills_per_trap = westland_stats.NUM_KILLS.sum()/westland_stats.NUM_TRAPS.sum()
westland_kills_per_trap

np.float64(2.0115942028985505)

In [ ]:
query_4 = """
WITH ranked_stats AS (SELECT
*
FROM `goodnature.fct_nz_annual_stats`
WHERE TERRITORIAL_AUTHORITY = 'Southland District'
)
SELECT
YEAR,
TERRITORY_AUTHORITY_ID,
TERRITORIAL_AUTHORITY,
NUM_SUBURBS,
TOTAL_NUM_SUBURBS,
SUBURB_COVERAGE_PERCENTAGE,
NUM_TOWNS,
TOTAL_NUM_TOWNS,
TOWN_COVERAGE_PERCENTAGE,
NUM_KILLS,
NUM_TRAPS
FROM ranked_stats
ORDER BY YEAR
"""

query_job4 = client.query(query=query_4)
southland_stats = query_job4.to_dataframe()
southland_stats

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,YEAR,TERRITORY_AUTHORITY_ID,TERRITORIAL_AUTHORITY,NUM_SUBURBS,TOTAL_NUM_SUBURBS,SUBURB_COVERAGE_PERCENTAGE,NUM_TOWNS,TOTAL_NUM_TOWNS,TOWN_COVERAGE_PERCENTAGE,NUM_KILLS,NUM_TRAPS
0,2017,73,Southland District,2,229,0.873362,2,34,5.882353,7,3
1,2018,73,Southland District,0,229,0.000000,0,34,0.000000,0,0
2,2019,73,Southland District,17,229,7.423581,11,34,32.352941,1092,984
3,2020,73,Southland District,62,229,27.074236,21,34,61.764706,3258,2614
4,2021,73,Southland District,33,229,14.410480,17,34,50.000000,356,298


In [55]:
southland_kills_per_trap = southland_stats.NUM_KILLS.sum()/southland_stats.NUM_TRAPS.sum()
southland_kills_per_trap

np.float64(1.2087714798666325)

In [ ]:
query_5 = """

WITH year_quarter_stats AS (
SELECT FORMAT_DATE('%Y-Q%Q', DATE(strike_at)) AS year_quarter, *
FROM `goodnature.stg_trap_data`
WHERE activity_type = 'STRIKE' AND latitude = -45.6060987
AND longitude = 167.3607653 AND strike_at > '2019-01-01'
ORDER BY strike_at)
SELECT year_quarter,
COUNT(strike_at) AS num_kills,
COUNT(DISTINCT trap_id) AS num_traps
FROM year_quarter_stats
GROUP BY year_quarter
ORDER BY year_quarter;
"""

query_job5 = client.query(query=query_5)
fiordland_stats = query_job5.to_dataframe()
fiordland_stats

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,year_quarter,num_kills,num_traps
0,2019-Q2,9,8
1,2019-Q3,78,60
2,2019-Q4,747,213
3,2020-Q1,1613,304
4,2020-Q2,517,178
5,2020-Q3,303,98
6,2020-Q4,54,33
7,2021-Q1,70,55
8,2021-Q2,2,1


In [61]:
fiordland_fig = go.Figure()
fiordland_fig.add_trace(
    go.Bar(
        x=fiordland_stats["year_quarter"],
        y=fiordland_stats["num_kills"],
        name="Number of Kills",
        marker_color='indianred',
        text=fiordland_stats["num_kills"]
    )
)
fiordland_fig.add_trace(
    go.Bar(
        x=fiordland_stats["year_quarter"],
        y=fiordland_stats["num_traps"],
        name="Number of Traps",
        marker_color='lightsalmon',
        text=fiordland_stats["num_traps"]
    )
)
fiordland_fig.update_layout(
    barmode="group",
    xaxis_tickangle=45,
    title="Number of kills and number of active traps in each quarter since 2019",
    xaxis_title="Quarters",
    yaxis_title="Number of kills/traps")
fiordland_fig.show()

In [ ]:
query_6 = """
WITH top_ten_quarterly AS (
SELECT
ROW_NUMBER() OVER(PARTITION BY YEAR_QUARTER ORDER BY NUM_KILLS DESC, NUM_TRAPS DESC) AS RANK_ORDER,
*
FROM `goodnature.fct_nz_quarterly_stats`
)
SELECT
CONCAT(YEAR_QUARTER, '-', FORMAT('%02d', RANK_ORDER)) AS QUARTERLY_RANK,
*
FROM top_ten_quarterly
WHERE RANK_ORDER <= 10
ORDER BY QUARTERLY_RANK;
"""

query_job6 = client.query(query=query_6)
quarterly_stat = query_job6.to_dataframe()
quarterly_stat

/Users/jacktoke/Codes/QUT/qut-data-analytics-duckdb-final/.venv/lib/python3.12/site-packages/google/cloud/bigquery/table.py:1994: UserWarning:

BigQuery Storage module not found, fetch data with the REST endpoint instead.



,QUARTERLY_RANK,RANK_ORDER,YEAR_QUARTER,full_address,address_number,full_road_name,suburb_locality,town_city,territorial_authority,latitude,longitude,NUM_KILLS,NUM_TRAPS
0,2017-Q2-01,1,2017-Q2,"12 Louise Lane, Waikawau, Coromandel",12,Louise Lane,Waikawau,Coromandel,Thames-Coromandel District,-36.602906,175.542580,107,30
1,2017-Q2-02,2,2017-Q2,"56 Arran Bay Road, Waiheke Island",56,Arran Bay Road,Waiheke Island,Waiheke Island,Auckland,-36.818333,175.162719,93,7
2,2017-Q2-03,3,2017-Q2,"111 Hornsey Road, Melrose, Wellington",111,Hornsey Road,Melrose,Wellington,Wellington City,-41.331002,174.787248,74,4
3,2017-Q2-04,4,2017-Q2,"595D Colville Road, Amodeo Bay, Coromandel",595D,Colville Road,Amodeo Bay,Coromandel,Thames-Coromandel District,-36.724129,175.442425,59,19
4,2017-Q2-05,5,2017-Q2,"151 Houghton Bay Road, Houghton Bay, Wellington",151,Houghton Bay Road,Houghton Bay,Wellington,Wellington City,-41.337342,174.787560,46,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2021-Q3-06,6,2021-Q3,"11 Peary Road, Mount Eden, Auckland",11,Peary Road,Mount Eden,Auckland,Auckland,-36.895137,174.756722,3,1
168,2021-Q3-07,7,2021-Q3,"2654 Christchurch Akaroa Road, Ataahua, Tai Tapu",2654,Christchurch Akaroa Road,Ataahua,Tai Tapu,Christchurch City,-43.780777,172.658361,3,1
169,2021-Q3-08,8,2021-Q3,"34 Marram Way, Peka Peka, Waikanae",34,Marram Way,Peka Peka,Waikanae,Kapiti Coast District,-40.838153,175.051142,3,1
170,2021-Q3-09,9,2021-Q3,"3 Captain Cook Road, Cooks Beach, Whitianga",3,Captain Cook Road,Cooks Beach,Whitianga,Thames-Coromandel District,-36.838121,175.739945,3,1


In [108]:
import plotly.express as px

quarterly_stat["TEXT"] = "Kills: " + quarterly_stat["NUM_KILLS"].astype(str) + ", Traps: " + quarterly_stat["NUM_TRAPS"].astype(str)
territories = sorted(quarterly_stat["territorial_authority"].unique())
colours = [
    "#69142C",
    "#08819D",
    "#E3D5AB",
    "#147A4B",
    "#6A80FA",
    "#957F05",
    "#4361C4",
    "#7B7C3B",
    "#84CA75",
    "#4C6A72",
    "#75D414",
    "#B9DA25",
    "#8AF971",
    "#390304",
    "#904ABE",
    "#4625DA",
    "#F909B2",
    "#1C2A54",
    "#8483C4",
    "#49676B",
    "#1433CA",
    "#59596A",
    "#A972F8",
    "#8A2BEB",
    "#B3958D",
    "#B69894",
    "#F77E62",
    "#568D07",
    "#AAD542",
    "#7A398F",
    "#0CC40C",
    "#6FB541",
    "#06F64D",
    "#85C670",
    "#C70AF8",
    "#DF96DD",
    "#BC9E3B",
    "#A6A695",
    "#EF7E54",
    "#F33BF3",
    "#EB85B4",
    "#552ABD",
    "#206922",
    "#96EBD3",
    "#EBCC35",
    "#1081AB",
    "#C6FCFB",
    "#75068E",
    "#38F507",
    "#7B358A"
]

quarterly_ranks = sorted(quarterly_stat["QUARTERLY_RANK"])
quarterly_stat["ORDERED_RANK"] = pd.Categorical(quarterly_stat["QUARTERLY_RANK"], categories=quarterly_ranks, ordered=True)

quarterly_fig = px.bar(quarterly_stat,
                       y="NUM_KILLS",
                       x="QUARTERLY_RANK",
                       text="TEXT",
                       color="territorial_authority",
                       hover_data=["full_address", "territorial_authority", "NUM_KILLS", "NUM_TRAPS"],
                       color_discrete_sequence=colours,
                     category_orders={"QUARTERLY_RANK": quarterly_ranks},
                       )
quarterly_fig.update_traces(textposition="outside")
quarterly_fig.update_layout(uniformtext_minsize=8, uniformtext_mode="hide", height=1000)
quarterly_fig.show()

,QUARTERLY_RANK,RANK_ORDER,YEAR_QUARTER,full_address,address_number,full_road_name,suburb_locality,town_city,territorial_authority,latitude,longitude,NUM_KILLS,NUM_TRAPS,TEXT,TEST
0,2017-Q2-01,1,2017-Q2,"12 Louise Lane, Waikawau, Coromandel",12,Louise Lane,Waikawau,Coromandel,Thames-Coromandel District,-36.602906,175.542580,107,30,"Kills: 107, Traps: 30",2017-Q2-01
1,2017-Q2-02,2,2017-Q2,"56 Arran Bay Road, Waiheke Island",56,Arran Bay Road,Waiheke Island,Waiheke Island,Auckland,-36.818333,175.162719,93,7,"Kills: 93, Traps: 7",2017-Q2-02
2,2017-Q2-03,3,2017-Q2,"111 Hornsey Road, Melrose, Wellington",111,Hornsey Road,Melrose,Wellington,Wellington City,-41.331002,174.787248,74,4,"Kills: 74, Traps: 4",2017-Q2-03
3,2017-Q2-04,4,2017-Q2,"595D Colville Road, Amodeo Bay, Coromandel",595D,Colville Road,Amodeo Bay,Coromandel,Thames-Coromandel District,-36.724129,175.442425,59,19,"Kills: 59, Traps: 19",2017-Q2-04
4,2017-Q2-05,5,2017-Q2,"151 Houghton Bay Road, Houghton Bay, Wellington",151,Houghton Bay Road,Houghton Bay,Wellington,Wellington City,-41.337342,174.787560,46,1,"Kills: 46, Traps: 1",2017-Q2-05
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
167,2021-Q3-06,6,2021-Q3,"11 Peary Road, Mount Eden, Auckland",11,Peary Road,Mount Eden,Auckland,Auckland,-36.895137,174.756722,3,1,"Kills: 3, Traps: 1",2021-Q3-06
168,2021-Q3-07,7,2021-Q3,"2654 Christchurch Akaroa Road, Ataahua, Tai Tapu",2654,Christchurch Akaroa Road,Ataahua,Tai Tapu,Christchurch City,-43.780777,172.658361,3,1,"Kills: 3, Traps: 1",2021-Q3-07
169,2021-Q3-08,8,2021-Q3,"34 Marram Way, Peka Peka, Waikanae",34,Marram Way,Peka Peka,Waikanae,Kapiti Coast District,-40.838153,175.051142,3,1,"Kills: 3, Traps: 1",2021-Q3-08
170,2021-Q3-09,9,2021-Q3,"3 Captain Cook Road, Cooks Beach, Whitianga",3,Captain Cook Road,Cooks Beach,Whitianga,Thames-Coromandel District,-36.838121,175.739945,3,1,"Kills: 3, Traps: 1",2021-Q3-09


In [ ]:
# -39.112449, 173.919373
# -39.386351, 174.173492